In [1]:
from utils import css_from_file
css_from_file('style/style.css')

In [2]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/home/ramya/.pyenv/versions/miniconda3-latest/envs/dsr/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Read data from the file __data/boehringer/(train|test).csv__ from the data folder. 

The first column is a binary variable that you want to predict. The rest are numericals

In [3]:
def load(path):
    df = pd.read_csv(path)
    if "Activity" not in df.columns:
        df["Activity"] = np.nan
    return df.drop("Activity",axis=1), df.Activity
    
X_tr, y_tr = load("data/boehringer/train.csv")
X_te, y_te = load("data/boehringer/test.csv")

print("training data shape", X_tr.shape)
print("testing data shape", X_te.shape)

training data shape (3751, 1776)
testing data shape (2501, 1776)


In [4]:
X_tr.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,0.243144,...,0,0,0,0,0,0,0,0,0,0
1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,0.106480,...,1,1,1,1,0,1,0,0,1,0
2,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,0.352308,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,0.208989,...,0,0,0,0,0,0,0,0,0,0
4,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,0.125177,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_tr.describe()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,0.270411,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,0.096128,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,0.003040,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,0.194357,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,0.284316,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,0.344626,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
list(X_tr.columns)

['D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'D16',
 'D17',
 'D18',
 'D19',
 'D20',
 'D21',
 'D22',
 'D23',
 'D24',
 'D25',
 'D26',
 'D27',
 'D28',
 'D29',
 'D30',
 'D31',
 'D32',
 'D33',
 'D34',
 'D35',
 'D36',
 'D37',
 'D38',
 'D39',
 'D40',
 'D41',
 'D42',
 'D43',
 'D44',
 'D45',
 'D46',
 'D47',
 'D48',
 'D49',
 'D50',
 'D51',
 'D52',
 'D53',
 'D54',
 'D55',
 'D56',
 'D57',
 'D58',
 'D59',
 'D60',
 'D61',
 'D62',
 'D63',
 'D64',
 'D65',
 'D66',
 'D67',
 'D68',
 'D69',
 'D70',
 'D71',
 'D72',
 'D73',
 'D74',
 'D75',
 'D76',
 'D77',
 'D78',
 'D79',
 'D80',
 'D81',
 'D82',
 'D83',
 'D84',
 'D85',
 'D86',
 'D87',
 'D88',
 'D89',
 'D90',
 'D91',
 'D92',
 'D93',
 'D94',
 'D95',
 'D96',
 'D97',
 'D98',
 'D99',
 'D100',
 'D101',
 'D102',
 'D103',
 'D104',
 'D105',
 'D106',
 'D107',
 'D108',
 'D109',
 'D110',
 'D111',
 'D112',
 'D113',
 'D114',
 'D115',
 'D116',
 'D117',
 'D118',
 'D119',
 'D120',
 'D121',
 'D122',
 'D123',
 

Exercise
---------------------

Using the starter code below try to improve the solution

1. What kind of models you can use?
2. Try changing model parameters to get the best cross validation error.
3. Use pipeline to transform features before modeling:
   - use some feature selection mechanism
   - use dimension reduction method (pca, svd, etc)
   
Tip: It is ok to loop over models and datasets like this.

```python
for data in [pipeline_1, pipeline_2, pipeline_3]:
    for model in [model_1, model_2, model_3]:
        # do stuff
```

In [7]:
from sklearn.ensemble import RandomForestClassifier
from cross_validation import cross_val_apply

clf = RandomForestClassifier(n_estimators=10,n_jobs=-1)

%time oof_predictions = cross_val_apply(clf, X_tr, y_tr, decision_func="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

CPU times: user 768 ms, sys: 16 ms, total: 784 ms
Wall time: 803 ms
Your error is 0.829336276397
You can still improve :)


In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.cross_validation import cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

    
from sklearn.preprocessing import Imputer



from sklearn.ensemble import RandomForestClassifier
from cross_validation import cross_val_apply

clf = RandomForestClassifier(n_estimators=10,n_jobs=-1)

%time oof_predictions = cross_val_apply(clf, X_tr, y_tr, decision_func="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")
    



CPU times: user 784 ms, sys: 12 ms, total: 796 ms
Wall time: 811 ms
Your error is 0.728895385748
You can still improve :)


In [11]:
processing_pipeline = make_pipeline(
    PCA(n_components= 100),
    LogisticRegression(),
        )


%time oof_predictions = cross_val_apply(processing_pipeline, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")
    

        

CPU times: user 332 ms, sys: 40 ms, total: 372 ms
Wall time: 7.5 s
Your error is 0.525507326897
You can still improve :)


In [12]:
processing_pipeline2 = make_pipeline(
    PCA(n_components= 100),
    XGBClassifier(),
        )


%time oof_predictions = cross_val_apply(processing_pipeline2, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")
    

CPU times: user 356 ms, sys: 68 ms, total: 424 ms
Wall time: 1min 31s
Your error is 0.514220954978
You can still improve :)


In [13]:
processing_pipeline3 = make_pipeline(
    StandardScaler(),
    Imputer(),
    PCA(n_components= 100),
    XGBClassifier(),
        )


%time oof_predictions = cross_val_apply(processing_pipeline3, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

CPU times: user 376 ms, sys: 60 ms, total: 436 ms
Wall time: 1min 31s
Your error is 0.509298530113
You can still improve :)


In [14]:
processing_pipeline4 = make_pipeline(
    MinMaxScaler(),
    Imputer(),
    PCA(n_components= 100),
    XGBClassifier(),
        )


%time oof_predictions = cross_val_apply(processing_pipeline4, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

CPU times: user 356 ms, sys: 80 ms, total: 436 ms
Wall time: 1min 31s
Your error is 0.518484864269
You can still improve :)


In [15]:
processing_pipeline4 = make_pipeline(
    StandardScaler(),
    Imputer(),
    PCA(n_components= 300),
    XGBClassifier(),
        )


%time oof_predictions = cross_val_apply(processing_pipeline4, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

CPU times: user 364 ms, sys: 76 ms, total: 440 ms
Wall time: 1min 38s
Your error is 0.506457092632
You can still improve :)


In [21]:
processing_pipeline4 = make_pipeline(
    StandardScaler(),
    Imputer(),
    PCA(n_components= 100),
    LogisticRegression(),
        )

%time oof_predictions = cross_val_apply(processing_pipeline4, X_tr, y_tr, decision_func="predict_proba", n_jobs= -1)

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
if err > 0.5:
    print("You can still improve :)")

CPU times: user 312 ms, sys: 60 ms, total: 372 ms
Wall time: 7.92 s
Your error is 0.5475514328
You can still improve :)
